In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## We will scrape the web from wikipedia using urllib and BeautifulSoup

In [3]:
## We will scrape the web from wikipedia using urllib and BeautifulSoup

import urllib.request
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)


# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")
print(soup.prettify())



<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"86896ea0-f7e9-4867-b334-279f0aa0c23f","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":960187814,"wgRevisionId":960187814,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [4]:
## The table we want has a class called "wikitable sortable"

right_table=soup.find('table', class_='wikitable sortable')
right_table

<table class="wikitable sortable">
<tbody><tr>
<th>Postal Code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>
<tr>
<td>M1A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A
</td>
<td>North York
</td>
<td>Parkwoods
</td></tr>
<tr>
<td>M4A
</td>
<td>North York
</td>
<td>Victoria Village
</td></tr>
<tr>
<td>M5A
</td>
<td>Downtown Toronto
</td>
<td>Regent Park, Harbourfront
</td></tr>
<tr>
<td>M6A
</td>
<td>North York
</td>
<td>Lawrence Manor, Lawrence Heights
</td></tr>
<tr>
<td>M7A
</td>
<td>Downtown Toronto
</td>
<td>Queen's Park, Ontario Provincial Government
</td></tr>
<tr>
<td>M8A
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M9A
</td>
<td>Etobicoke
</td>
<td>Islington Avenue, Humber Valley Village
</td></tr>
<tr>
<td>M1B
</td>
<td>Scarborough
</td>
<td>Malvern, Rouge
</td></tr>
<tr>
<td>M2B
</td>
<td>Not assigned
</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3B
</td>
<td>

In [5]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))


In [6]:
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough'] = B
df['Neighborhood'] = C
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


## As every word is finished with \n we are going to cut them

In [7]:
## As every word is finished with \n we are going to cut them

A1 = []
B1 = []
C1 = []

for word in A:
    result = word.find('\n')
    word = word[:result]
    A1.append(word)
    
for word in B:
    result = word.find('\n')
    word = word[:result]
    B1.append(word)
    
for word in C:
    result = word.find('\n')
    word = word[:result]
    C1.append(word)
    
df=pd.DataFrame(A1,columns=['Postal Code'])
df['Borough'] = B1
df['Neighborhood'] = C1
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [8]:
df.shape

(180, 3)

## We are going to look if there are rows having not assigned neighborhoods, but assigned Boroughs. And vice versa

In [9]:
## We are going to look if there are rows having not assigned neighborhoods, but assigned Boroughs. And vice versa

filter1 = (df['Borough'] != 'Not assigned') & (df['Neighborhood'] == 'Not assigned')
filter2 = (df['Borough'] == 'Not assigned') & (df['Neighborhood'] != 'Not assigned')

df1 = df[filter1]
print('df1 Shape is', df1.shape)

df2 = df[filter2]
print('df2 Shape is', df2.shape)

df1 Shape is (0, 3)
df2 Shape is (0, 3)


#### Result from above shows that there aren't

## We apply now a filter to drop those "not assigned" rows

In [10]:
## We apply now filter to drop those "not assigned" rows

filter = ((df['Borough'] != 'Not assigned') | (df['Neighborhood'] != 'Not assigned')) 
df = df[filter]

print('Filtered Dataframe Shape is', df.shape)

Filtered Dataframe Shape is (103, 3)


## We look for duplicates in the POSTAL CODE ; to know if we have to merge some neighborhoods

In [178]:
## Descending Order. So if there's a duplicate Postal Code it should be in first place

print(df['Postal Code'].value_counts().sort_values(ascending=False).head())

M7R    1
M2P    1
M3M    1
M4L    1
M4W    1
Name: Postal Code, dtype: int64


In [11]:
print(df['Borough'].value_counts().sort_values(ascending=False).head(10))

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East York            5
East Toronto         5
Mississauga          1
Name: Borough, dtype: int64


### How many Boroughs and how many Neighborhoods

In [12]:
### How many Boroughs and how many Neighborhoods

print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

col = ['Borough','Neighborhood']

df[col].groupby('Borough').count()

The dataframe has 10 boroughs and 103 neighborhoods.


,Neighborhood
Borough,
Central Toronto,9
Downtown Toronto,19
East Toronto,5
East York,5
Etobicoke,12
Mississauga,1
North York,24
Scarborough,17
West Toronto,6


## We look for latitude and longitude from the link. Then we are going to merge using the postal code

In [13]:
## We look for latitude and longitude from the link showed in the evaluation. Then we are going to merge using the postal code
LAT_LON = pd.read_csv('http://cocl.us/Geospatial_data')
LAT_LON

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [14]:
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [15]:
#left.merge(right, on='user_id', how='left')

df_merged = df.merge(LAT_LON, on ='Postal Code', how='left')
print(df_merged.head(10))
print(df.head(10))

  Postal Code           Borough                                 Neighborhood  \
0         M3A        North York                                    Parkwoods   
1         M4A        North York                             Victoria Village   
2         M5A  Downtown Toronto                    Regent Park, Harbourfront   
3         M6A        North York             Lawrence Manor, Lawrence Heights   
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government   
5         M9A         Etobicoke      Islington Avenue, Humber Valley Village   
6         M1B       Scarborough                               Malvern, Rouge   
7         M3B        North York                                    Don Mills   
8         M4B         East York              Parkview Hill, Woodbine Gardens   
9         M5B  Downtown Toronto                     Garden District, Ryerson   

    Latitude  Longitude  
0  43.753259 -79.329656  
1  43.725882 -79.315572  
2  43.654260 -79.360636  
3  43.718518 -7

#### We will apply a filter for those Boroughs containing the word "Toronto", and also Etobicoke ( random election )

In [22]:
df_merged.head()
df_merged['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [117]:
df_f = df_merged
#filter1 = (df_f['Borough'] == 'Etobicoke') | (df_f['Borough'] == 'Central Toronto') | (df_f['Borough'] == 'York') | (df_f['Borough'] == 'Downtown Toronto') | (df_f['Borough'] == 'East Toronto') | (df_f['Borough'] == 'West Toronto')
#df_f = df_f[filter1]
df_f = df_f[df_f['Borough'].str.contains("Toronto")]
df_f['Borough'].value_counts()

Downtown Toronto    19
Central Toronto      9
West Toronto         6
East Toronto         5
Name: Borough, dtype: int64

##### We can also filter and take some random Boroughs, like North York, Downtown Toronto, Etobicoke, Central Toronto and West Toronto

In [27]:
df_f.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724


### Now we start using maps and clustering

First we show the map and points using Folium

In [28]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Toronto are 43.6534817, -79.3839347.


In [118]:
# create map of Toronot using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

df = df_f

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [34]:
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650943,-79.554724


## We define Foursquare credentials to explore venues and then cluster the neighborhoods

In [73]:
CLIENT_ID = 'P1PBTBTEGIEHDJYKQT1ZAGKFTV0NUIHWATIU4IJLZHINPFP4' # your Foursquare ID
CLIENT_SECRET = 'WQJIDPHICC0BM12PKXVRDFIRQ0RPXGXM1H5OUD0GPAXSS23H' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: P1PBTBTEGIEHDJYKQT1ZAGKFTV0NUIHWATIU4IJLZHINPFP4
CLIENT_SECRET:WQJIDPHICC0BM12PKXVRDFIRQ0RPXGXM1H5OUD0GPAXSS23H


In [119]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

## Function that get venues for every neighborhood

In [120]:
LIMIT=100

def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


Let's check the size of the resulting DF

In [122]:
print(toronto_venues.shape)
toronto_venues.head()

(2113, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Regent Park, Harbourfront",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park


In [123]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,92,92,92,92,92,92
"Brockton, Parkdale Village, Exhibition Place",41,41,41,41,41,41
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",22,22,22,22,22,22
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,100,100,100,100,100,100
Christie,19,19,19,19,19,19
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,46,46,46,46,46,46


#### Let's find out how many unique categories can be curated from all the returned venues

In [124]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 257 uniques categories.


### Start analysis by neighborhood

In [125]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

toronto_onehot.shape

(2113, 257)

In [126]:
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [127]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Plane,Playground,Plaza,Poke Place,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Tram Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Berczy Park,0.010870,0.00,0.0000,0.0000,0.000,0.000,0.000,0.010870,0.000000,0.000000,0.010870,0.00,0.00,0.010870,0.00,0.000000,0.000000,0.000000,0.010870,0.000000,0.010870,0.021739,0.000000,0.010870,0.00,0.01087,0.01087,0.00,0.021739,0.000000,0.00,0.000000,0.01087,0.000000,0.0000,0.000000,0.000000,0.00,0.021739,0.000000,0.000000,0.00,0

In [128]:
toronto_grouped.shape

(39, 257)

#### Let's print each neighborhood along with the top 5 most common venues

In [129]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0          Coffee Shop  0.09
1                 Café  0.04
2   Seafood Restaurant  0.04
3                Hotel  0.03
4  Japanese Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0            Café  0.07
1     Coffee Shop  0.07
2  Breakfast Spot  0.05
3      Restaurant  0.05
4  Sandwich Place  0.05


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
                    venue  freq
0           Burrito Place  0.09
1                    Park  0.09
2             Yoga Studio  0.05
3  Thrift / Vintage Store  0.05
4              Steakhouse  0.05


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0       Coffee Shop  0.12
1    Airport Lounge  0.12
2   Airport Service  0.12
3  Airport Terminal  0.12
4   Harbor / Marina  0.06


----Central Bay Street----
                 v

#### Let's put it into a Pandas DF, first writing a function to sort them in descending order

In [130]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [131]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Seafood Restaurant,Café,Japanese Restaurant,Restaurant,Cocktail Bar,Hotel,Breakfast Spot,Pub,Cheese Shop
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Gift Shop,Breakfast Spot,Sandwich Place,Restaurant,Bakery,Supermarket,Stadium,Snack Place
2,"Business reply mail Processing Centre, South C...",Park,Burrito Place,Bakery,Comic Shop,Pizza Place,Gym / Fitness Center,Clothing Store,Restaurant,Sandwich Place,Skate Park
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Bar,Sculpture Garden,Rental Car Location,Harbor / Marina,Plane
4,Central Bay Street,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Thai Restaurant,Bar,Sushi Restaurant,Chinese Restaurant


##  Cluster Neighborhoods

In [135]:
# set number of clusters
kclusters = 6

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=3).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 5, 1, 1, 2, 2, 2, 0, 2, 3, 2,
       2, 1, 1, 1, 3, 4, 2, 1])

In [137]:
# add clustering labels

neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

#toronto_merged =

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = df_f.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Theater,Bakery,Pub,Park,Café,Breakfast Spot,Mediterranean Restaurant,Liquor Store,French Restaurant
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sandwich Place,Falafel Restaurant,Sushi Restaurant,Burrito Place,Café,Park,Diner,Italian Restaurant,Poke Place
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Burger Joint,Tea Room,Bubble Tea Shop,Japanese Restaurant,Chinese Restaurant,Electronics Store,Bookstore,Falafel Restaurant
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Restaurant,Gastropub,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Bakery
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Gastropub,Women's Store,Toy / Game Store,Ramen Restaurant,Dance Studio,Mexican Restaurant,Breakfast Spot,Bakery,French Restaurant


In [138]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

rainbow

['#8000ff', '#1996f3', '#4df3ce', '#b2f396', '#ff964f', '#ff0000']

In [139]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [140]:
toronto_merged['Cluster Labels'].value_counts()

2    19
1    15
3     2
5     1
4     1
0     1
Name: Cluster Labels, dtype: int64

### Cluster 0

In [116]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue
11,Etobicoke,0,Pizza Place,Chinese Restaurant,Tea Room,Deli / Bodega,Event Space,Electronics Store,Eastern European Restaurant,Drugstore,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
70,Etobicoke,0,Pizza Place,Ice Cream Shop,Intersection,Discount Store,Middle Eastern Restaurant,Chinese Restaurant,Flea Market,Sandwich Place,Coffee Shop,Dog Run,Doner Restaurant,Donut Shop
77,Etobicoke,0,Pharmacy,Beer Store,Mobile Phone Shop,Bus Line,Chinese Restaurant,Sandwich Place,Pizza Place,Donut Shop,Doner Restaurant,Department Store,Distribution Center,Discount Store
89,Etobicoke,0,Pizza Place,Grocery Store,Fried Chicken Joint,Sandwich Place,Fast Food Restaurant,Beer Store,Japanese Restaurant,Pharmacy,Gastropub,Dance Studio,Drugstore,Donut Shop
93,Etobicoke,0,Pizza Place,Skating Rink,Dance Studio,Pharmacy,Coffee Shop,Pub,Sandwich Place,Gym,Animal Shelter,Farm,Event Space,Electronics Store


### Cluster 1

In [141]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Downtown Toronto,1,Coffee Shop,Theater,Bakery,Pub,Park,Café,Breakfast Spot,Mediterranean Restaurant,Liquor Store,French Restaurant
4,Downtown Toronto,1,Coffee Shop,Sandwich Place,Falafel Restaurant,Sushi Restaurant,Burrito Place,Café,Park,Diner,Italian Restaurant,Poke Place
9,Downtown Toronto,1,Coffee Shop,Clothing Store,Burger Joint,Tea Room,Bubble Tea Shop,Japanese Restaurant,Chinese Restaurant,Electronics Store,Bookstore,Falafel Restaurant
15,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Gastropub,Cosmetics Shop,Italian Restaurant,Seafood Restaurant,Japanese Restaurant,American Restaurant,Bakery
20,Downtown Toronto,1,Coffee Shop,Seafood Restaurant,Café,Japanese Restaurant,Restaurant,Cocktail Bar,Hotel,Breakfast Spot,Pub,Cheese Shop
24,Downtown Toronto,1,Coffee Shop,Café,Japanese Restaurant,Italian Restaurant,Bubble Tea Shop,Sandwich Place,Thai Restaurant,Bar,Sushi Restaurant,Chinese Restaurant
30,Downtown Toronto,1,Coffee Shop,Café,Restaurant,Hotel,Gym,Theater,Japanese Restaurant,Sushi Restaurant,Bookstore,Concert Hall
36,Downtown Toronto,1,Coffee Shop,Hotel,Park,Brewery,Sandwich Place,Café,Plaza,Pizza Place,Ice Cream Shop,Steakhouse
42,Downtown Toronto,1,Hotel,Coffee Shop,Café,Restaurant,American Restaurant,Concert Hall,Seafood Restaurant,Gym,Japanese Restaurant,Vegetarian / Vegan Restaurant
48,Downtown Toronto,1,Coffee Shop,Restaurant,Café,Hotel,Gym,Japanese Restaurant,Seafood Restaurant,American Restaurant,Italian Restaurant,Bakery


### Cluster 2

In [142]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,East Toronto,2,Pub,Gastropub,Women's Store,Toy / Game Store,Ramen Restaurant,Dance Studio,Mexican Restaurant,Breakfast Spot,Bakery,French Restaurant
25,Downtown Toronto,2,Grocery Store,Café,Coffee Shop,Park,Athletics & Sports,Candy Store,Baby Store,Italian Restaurant,Nightclub,Restaurant
31,West Toronto,2,Pharmacy,Bakery,Park,Pet Store,Coffee Shop,Music Venue,Discount Store,Café,Middle Eastern Restaurant,Brewery
37,West Toronto,2,Bar,Restaurant,Theater,Café,Vietnamese Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Cocktail Bar
41,East Toronto,2,Greek Restaurant,Coffee Shop,Pub,Italian Restaurant,Café,Ice Cream Shop,Bank,Furniture / Home Store,Bookstore,Restaurant
43,West Toronto,2,Café,Coffee Shop,Gift Shop,Breakfast Spot,Sandwich Place,Restaurant,Bakery,Supermarket,Stadium,Snack Place
47,East Toronto,2,Sandwich Place,Gym,Fast Food Restaurant,Café,Grocery Store,Liquor Store,Sushi Restaurant,Steakhouse,Fish & Chips Shop,Park
54,East Toronto,2,Café,Bar,Diner,American Restaurant,Coffee Shop,Bakery,Brewery,Vietnamese Restaurant,Gastropub,Sandwich Place
67,Central Toronto,2,Pizza Place,Hotel,Park,Sandwich Place,Deli / Bodega,Food & Drink Shop,Sushi Restaurant,Brewery,Café,Breakfast Spot
69,West Toronto,2,Café,Italian Restaurant,Mexican Restaurant,Grocery Store,Antique Shop,Thai Restaurant,Flea Market,Furniture / Home Store,Sushi Restaurant,Bar


### Cluster 3

In [143]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
83,Central Toronto,3,Park,Tennis Court,Gym,Women's Store,Donut Shop,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Eastern European Restaurant
91,Downtown Toronto,3,Park,Playground,Trail,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dessert Shop


### Cluster 4

In [144]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,4,Playground,Home Service,Garden,Farmers Market,Farm,Falafel Restaurant,Fast Food Restaurant,Event Space,Diner,Eastern European Restaurant


### Cluster 5

In [145]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 5, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
68,Central Toronto,5,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Donut Shop,Distribution Center,Dive Bar,Dog Run,Doner Restaurant
